## Cell type Means

### 11-11-24 use Nov 2024 version 2d object and Ensembl ID TF's.
### This code generates the mean log normal values for each cell type
### Trimmed version with mean only, does not have percent of cells.
###
#### 03-19-24 revisions for review cycle.
#### Added in broad cell type and tissue mean calculations
###
#### 04-07-24 Now clean meta data has '_' in both cell_ontology and broad_cell class names
#### 04-18-24 Added percent non-zero




## General Import Functions

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import seaborn as sns
import anndata
import string
import gc
from anndata import read_h5ad
from anndata import read_csv
from pandas import DataFrame
import h5py



/hpc/user_apps/data.science/conda_envs/single-cell-base/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)


In [2]:
#data switches

ALL_GENES = False

DataSet = 'Full'  # means all tissues
TenXOnly = True   # don't use plate data
Raw = False       # use raw counts, not log normalized data

In [59]:
# Stuff going out
DATA_OUT_DIR = '/hpc/mydata/robert.jones/TFManuscript2GITHubPush/Results/'
META_DATA_CLEAN = 'TSP1_30_version2d_10X_SS_2024_11_12_metadata_CLEAN_'
DATA_CT = 'TSP1_30_by_CellType_Full_Version2d_Nov2024_'

if Raw:
    DATA_CT = DATA_CT + 'Raw_'
else:
    DATA_CT = DATA_CT + 'LogNorm_'

if TenXOnly:
    DATA_CT = DATA_CT + '10XOnly_'
else:
    DATA_CT = DATA_CT + '10XandSS2_'    
    
if ALL_GENES:
    DATA_CT_TABLE = 'AllGenes/' + DATA_CT + 'Mean_AllGenes.csv'
    META_DATA_OUT = 'AllGenes/' + META_DATA_CLEAN+'AllGenes.csv'
else:
    DATA_CT_TABLE = 'TFonly/' + DATA_CT + 'Mean_TFonly.csv'
    META_DATA_OUT = 'TFonly/' + META_DATA_CLEAN+'TFonly.csv'
print(DATA_CT_TABLE)
print(META_DATA_OUT)


TFonly/TSP1_30_by_CellType_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_TFonly.csv
TFonly/TSP1_30_version2d_10X_SS_2024_11_12_metadata_CLEAN_TFonly.csv


In [60]:
#Stuff coming in
SHARED_DATA_DIR = '/hpc/projects/tabula_sapiens/singlecell_transcriptomics/TSP1_30_Paper2_Version2d_Oct2024/full_object/'
DATA_IN_DIR = '/hpc/mydata/robert.jones/TFManuscript2GITHubPush/DataIn/'

TF_DATA = 'HumanTranscriptionFactors.csv'
TF_DATA_ENSEMBL = 'HumanTranscriptionFactorsEnsembl.csv'

META_DATA = 'TSP1_30_metadata_min200_2500_decontx_scvi_donorassay_version2d_20241112.csv'
DATA = 'TSP1_30_min200_2500_decontx_scvi_donorassay_version2d_20241126.h5ad' 

In [61]:
print('Directory with data in = ', SHARED_DATA_DIR)
print()
print('Data In = ', DATA)
print()
print('MetaData In = ', META_DATA)
print()
print('Directory for data out = ', DATA_OUT_DIR)
print()
print('Data Out = ', DATA_CT_TABLE)

Directory with data in =  /hpc/projects/tabula_sapiens/singlecell_transcriptomics/TSP1_30_Paper2_Version2d_Oct2024/full_object/

Data In =  TSP1_30_min200_2500_decontx_scvi_donorassay_version2d_20241126.h5ad

MetaData In =  TSP1_30_metadata_min200_2500_decontx_scvi_donorassay_version2d_20241112.csv

Directory for data out =  /hpc/mydata/robert.jones/TFManuscript2GITHubPush/Results/

Data Out =  TFonly/TSP1_30_by_CellType_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_TFonly.csv


In [62]:
metadata = pd.read_csv(SHARED_DATA_DIR+META_DATA)

/tmp/ipykernel_502761/3065039129.py:1: DtypeWarning: Columns (5,6,7,8,9,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(SHARED_DATA_DIR+META_DATA)


In [63]:
metadata10x = metadata[metadata.method != 'smartseq']
print(list(metadata10x.columns))

#print()
#print('Replicate', metadata10x.replicate.unique())
#print('Method', metadata10x.method.unique())
#print('Assay', metadata10x.assay.unique())
#print('Notes', metadata10x.notes.unique())

#print()
#display('SampleID', metadata10x.iloc[[0, 10000, 20000, 30000, 140000, 35000]][['sample_id', 'method', 'assay', 'notes', 'replicate']])

['Unnamed: 0', 'donor', 'tissue', 'anatomical_position', 'method', 'cdna_plate', 'library_plate', 'notes', 'cdna_well', 'old_index', 'assay', 'sample_id', 'replicate', '10X_run', '10X_barcode', 'ambient_removal', 'donor_method', 'donor_assay', 'donor_tissue', 'donor_tissue_assay', 'cell_ontology_class', 'cell_ontology_id', 'compartment', 'broad_cell_class', 'free_annotation', 'manually_annotated', 'published_2022', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ercc', 'pct_counts_ercc', '_scvi_batch', '_scvi_labels', 'scvi_leiden_donorassay_full', 'age', 'sex', 'ethnicity', 'sample_number']


# Read object and list of transcription factors

In [64]:
print(SHARED_DATA_DIR + DATA)
#adata = read_h5ad(SHARED_DATA_DIR + DATA)
adata


/hpc/projects/tabula_sapiens/singlecell_transcriptomics/TSP1_30_Paper2_Version2d_Oct2024/full_object/TSP1_30_min200_2500_decontx_scvi_donorassay_version2d_20241126.h5ad


AnnData object with n_obs × n_vars = 1136218 × 61806
    obs: 'donor', 'tissue', 'anatomical_position', 'method', 'cdna_plate', 'library_plate', 'notes', 'cdna_well', 'old_index', 'assay', 'sample_id', 'replicate', '10X_run', '10X_barcode', 'ambient_removal', 'donor_method', 'donor_assay', 'donor_tissue', 'donor_tissue_assay', 'cell_ontology_class', 'cell_ontology_id', 'compartment', 'broad_cell_class', 'free_annotation', 'manually_annotated', 'published_2022', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ercc', 'pct_counts_ercc', '_scvi_batch', '_scvi_labels', 'scvi_leiden_donorassay_full', 'age', 'sex', 'ethnicity', 'sample_number'
    var: 'ensembl_id', 'gene_symbol', 'genome', 'mt', 'ercc', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: '_scvi_manager_uuid', '_scvi_uuid', '_training_mode', 'age_colors', 'assay_colors', 'compartment_colors', 'donor_colors', 'leiden', 'log1p', 'method_color

In [65]:
if TenXOnly:
    adata = adata[adata.obs.method == '10X']

In [66]:
adata

View of AnnData object with n_obs × n_vars = 1093048 × 61806
    obs: 'donor', 'tissue', 'anatomical_position', 'method', 'cdna_plate', 'library_plate', 'notes', 'cdna_well', 'old_index', 'assay', 'sample_id', 'replicate', '10X_run', '10X_barcode', 'ambient_removal', 'donor_method', 'donor_assay', 'donor_tissue', 'donor_tissue_assay', 'cell_ontology_class', 'cell_ontology_id', 'compartment', 'broad_cell_class', 'free_annotation', 'manually_annotated', 'published_2022', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ercc', 'pct_counts_ercc', '_scvi_batch', '_scvi_labels', 'scvi_leiden_donorassay_full', 'age', 'sex', 'ethnicity', 'sample_number'
    var: 'ensembl_id', 'gene_symbol', 'genome', 'mt', 'ercc', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: '_scvi_manager_uuid', '_scvi_uuid', '_training_mode', 'age_colors', 'assay_colors', 'compartment_colors', 'donor_colors', 'leiden', 'log1p', 'meth

In [67]:
adata.obs.method.value_counts

<bound method IndexOpsMixin.value_counts of TSP27_FAT_MAT_10X3primev31_1_1_AAACGAAGTTGAAGTA    10X
TSP27_FAT_MAT_10X3primev31_1_1_AAAGTCCAGAGAGCGG    10X
TSP27_FAT_MAT_10X3primev31_1_1_AAAGTGAAGCTCGACC    10X
TSP27_FAT_MAT_10X3primev31_1_1_AACACACGTATGTCTG    10X
TSP27_FAT_MAT_10X3primev31_1_1_AACCTTTAGAGTAACT    10X
                                                  ... 
TSP25_FAT_Brown_10X_1_6_TTTGTTGCAAAGCACG           10X
TSP25_FAT_Brown_10X_1_6_TTTGTTGGTAAGTCAA           10X
TSP25_FAT_Brown_10X_1_6_TTTGTTGGTATACCCA           10X
TSP25_FAT_Brown_10X_1_6_TTTGTTGGTGTGTCCG           10X
TSP25_FAT_Brown_10X_1_6_TTTGTTGTCAAGCGTT           10X
Name: method, Length: 1093048, dtype: category
Categories (1, object): ['10X']>

In [68]:
adata.var.index.isin(['ENSG00000284976']).sum()

1

In [79]:
adataGene = adata[: , adata.var.index == 'ENSG00000284976'].copy()

In [80]:
adataGene.X.sum() 

0.0

In [81]:
adataNZ = adataGene[adataGene.X != 0, ]

In [82]:
print(adataNZ.var.index)
adataNZ.obs.assay.value_counts()

Index(['ENSG00000284976'], dtype='object', name='index')


Series([], Name: count, dtype: int64)

In [83]:
adata.var.index.isin(['CD99']).sum()

1

In [84]:
def mapTFsbyEnsemblIDlist():
    TF_data = pd.read_csv(DATA_IN_DIR+TF_DATA_ENSEMBL)

    adata_table = pd.DataFrame()
    adata_table['ensembl_id'] = adata.var.ensembl_id.str.split('.', expand=True)[0]
    adata_table['gene_symbol'] = adata.var.gene_symbol
    
    # slice to just TF's
    TF_adata_table = adata_table[adata_table.ensembl_id.isin(TF_data.TF_names)]
    TF_adata_table.reset_index()
    TF_adata_table.sort_values('ensembl_id')
    
    return(TF_adata_table)

In [85]:
mapTFsbyEnsemblIDlist()

,ensembl_id,gene_symbol
index,,
NFYA,ENSG00000001167,NFYA
ARX,ENSG00000004848,ARX
HOXA11,ENSG00000005073,HOXA11
MEOX1,ENSG00000005102,MEOX1
SOX8,ENSG00000005513,SOX8
...,...,...
PCGF2,ENSG00000277258,PCGF2
ZNF670,ENSG00000277462,ZNF670
ZNF8,ENSG00000278129,ZNF8


In [86]:
if ALL_GENES:
    TF_list =  list(adata.var.index)
else:   #just the TF's
    TF_list = mapTFsbyEnsemblIDlist().index
len(TF_list)

1637

## Clean up cell type names

In [87]:
# read metatdata that goes with the full objects
# modify cell types names to ensure they  match
metadata = pd.read_csv(SHARED_DATA_DIR+META_DATA)

metadata = metadata[metadata.method == '10X']

TF_rownames = list(metadata.cell_ontology_class)
TF_rownames = [sub.replace(' ', '_') for sub in TF_rownames]
TF_rownames = [sub.replace('-', '_') for sub in TF_rownames]
TF_rownames = [sub.replace(',', '_') for sub in TF_rownames]
metadata.cell_ontology_class = TF_rownames

TF_rownames = list(metadata.broad_cell_class)
TF_rownames = [sub.replace(' ', '_') for sub in TF_rownames]
TF_rownames = [sub.replace('-', '_') for sub in TF_rownames]
TF_rownames = [sub.replace(',', '_') for sub in TF_rownames]
metadata.broad_cell_class = TF_rownames

print('Meta data clean saved as ', DATA_OUT_DIR+META_DATA_OUT )
metadata.to_csv(DATA_OUT_DIR+META_DATA_OUT)


/tmp/ipykernel_502761/133360280.py:3: DtypeWarning: Columns (5,6,7,8,9,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(SHARED_DATA_DIR+META_DATA)


Meta data clean saved as  /hpc/mydata/robert.jones/TFManuscript2GITHubPush/Results/TFonly/TSP1_30_version2d_10X_SS_2024_11_12_metadata_CLEAN_TFonly.csv


## Make TF x Cell Type dataframe of mean expression

In [88]:
CellTypesList = list(adata.obs.cell_ontology_class.unique())
#CellTypesList = ['basal cell','keratinocyte']  #for debug
len(CellTypesList)

175

In [89]:
TF_df = DataFrame()

for CT in CellTypesList:
    CellsOfThisType = list(adata.obs[adata.obs.cell_ontology_class == CT].index)

    CT_adata = adata[CellsOfThisType,TF_list]
    if Raw:
        TF_means = CT_adata.layers['raw_counts'].mean(0)
    else:
        TF_means = CT_adata.layers['log_normalized'].mean(0)
        
    df = pd.DataFrame(TF_means, columns = TF_list, index = [CT])

    TF_df = pd.concat([TF_df,df])



In [90]:
#convert cell type names to indexes without ' ' and '-'
TF_rownames = list(TF_df.index)
TF_rownames = [sub.replace(' ', '_') for sub in TF_rownames]
TF_rownames = [sub.replace('-', '_') for sub in TF_rownames]
TF_rownames = [sub.replace(',', '_') for sub in TF_rownames]
TF_df.index = TF_rownames
TF_df


index,NFYA,ARX,HOXA11,MEOX1,SOX8,ZNF195,ZFX,YBX2,ZNF263,DLX6,...,SEBOX,ZNF280B,ZNF2,HNF1B,DACH1,PCGF2,ZNF670,ZNF8,ZNF229,NR2E3
macrophage,0.154053,0.000519,0.000398,0.002769,0.001229,0.083150,0.421626,0.000253,0.114945,0.000113,...,0.000000,0.009422,0.015929,0.001249,0.063781,0.019405,0.010195,0.020019,0.002689,0.001045
mesenchymal_stem_cell_of_adipose_tissue,0.052249,0.000000,0.019499,0.004737,0.020782,0.088510,0.255203,0.000088,0.092353,0.000391,...,0.000000,0.004263,0.015310,0.000074,0.024646,0.116857,0.009335,0.019603,0.004925,0.005257
mast_cell,0.086978,0.000000,0.000475,0.002189,0.002324,0.069971,0.276791,0.000142,0.056720,0.000000,...,0.000000,0.004928,0.012086,0.001421,0.377390,0.005018,0.017056,0.012939,0.002050,0.001533
neutrophil,0.279022,0.000018,0.000058,0.000715,0.000033,0.009683,0.354291,0.000035,0.209102,0.000018,...,0.000000,0.000845,0.002228,0.000495,1.051913,0.000536,0.000586,0.006057,0.000084,0.000296
fibroblast,0.079945,0.000084,0.063043,0.005107,0.010603,0.092513,0.305828,0.000116,0.084879,0.002213,...,0.000023,0.002570,0.020048,0.001000,0.111145,0.127053,0.018906,0.021896,0.002380,0.005355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kidney_epithelial_cell,0.037919,0.001467,0.000461,0.001310,0.000000,0.191532,0.239228,0.005082,0.056315,0.000232,...,0.000070,0.042522,0.016841,0.471793,0.096221,0.120260,0.016763,0.037138,0.013244,0.001868
retinal_pigment_epithelial_cell,0.019420,0.000000,0.000000,0.000000,0.000000,0.051691,0.192917,0.016006,0.066641,0.000000,...,0.000000,0.058852,0.039459,0.000000,0.019574,0.204707,0.000000,0.000000,0.020606,0.019311
keratinocyte,0.056283,0.000000,0.000000,0.000000,0.000000,0.111229,0.129412,0.000000,0.079990,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.079990,0.000000,0.000000,0.000000
enterocyte_of_epithelium_proper_of_jejunum,0.168485,0.000000,0.000000,0.000000,0.000000,0.180533,0.454409,0.109400,0.111378,0.000000,...,0.000000,0.006153,0.018718,0.190655,0.879147,0.023220,0.009989,0.014616,0.000000,0.008909


In [91]:
# check they are the same 
metaset = set(metadata.cell_ontology_class.unique())
TFdfset = set(TF_df.index)
print(metaset.difference(TFdfset))
print(TFdfset.difference(metaset))


set()
set()


In [92]:
print('TFxCT saved as ', DATA_OUT_DIR+DATA_CT_TABLE )
print('Meta data clean saved as ', DATA_OUT_DIR+META_DATA_OUT )

TF_df.to_csv(DATA_OUT_DIR+DATA_CT_TABLE)
metadata.to_csv(DATA_OUT_DIR+META_DATA_OUT)

TFxCT saved as  /hpc/mydata/robert.jones/TFManuscript2GITHubPush/Results/TFonly/TSP1_30_by_CellType_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_TFonly.csv
Meta data clean saved as  /hpc/mydata/robert.jones/TFManuscript2GITHubPush/Results/TFonly/TSP1_30_version2d_10X_SS_2024_11_12_metadata_CLEAN_TFonly.csv


## Make celltype percent expressing dataframe

In [93]:
if ALL_GENES:
    DATA_CT_TABLE = 'AllGenes/TSP1_30_by_PercentNZ_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_AllGenes.csv'
else:
    DATA_CT_TABLE = 'TFonly/TSP1_30_by_PercentNZ_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_TFonly.csv'

In [94]:
TF_frac = DataFrame()

for CT in CellTypesList:
    CellsOfThisType = adata.obs.index[adata.obs.cell_ontology_class == CT]

    NZarray = adata[CellsOfThisType, TF_list].layers['log_normalized'].toarray()
    NZarray_binarized = (NZarray > 0) * 1
    NZdf = pd.DataFrame(NZarray_binarized)

    df = pd.DataFrame(NZdf.sum()/len(CellsOfThisType))
    df.index = TF_list
    df.columns = [CT]
    
    TF_frac = pd.concat([TF_frac,df.T])


In [95]:
#convert cell type names to indexes without ' ' and '-'
TF_rownames = list(TF_frac.index)
TF_rownames = [sub.replace(' ', '_') for sub in TF_rownames]
TF_rownames = [sub.replace('-', '_') for sub in TF_rownames]
TF_rownames = [sub.replace(',', '_') for sub in TF_rownames]
TF_frac.index = TF_rownames

print(DATA_OUT_DIR+DATA_CT_TABLE)
TF_frac.to_csv(DATA_OUT_DIR+DATA_CT_TABLE)
TF_frac


/hpc/mydata/robert.jones/TFManuscript2GITHubPush/Results/TFonly/TSP1_30_by_PercentNZ_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_TFonly.csv


index,NFYA,ARX,HOXA11,MEOX1,SOX8,ZNF195,ZFX,YBX2,ZNF263,DLX6,...,SEBOX,ZNF280B,ZNF2,HNF1B,DACH1,PCGF2,ZNF670,ZNF8,ZNF229,NR2E3
macrophage,0.237442,0.000721,0.000856,0.004820,0.001847,0.159461,0.513087,0.000435,0.189945,0.000195,...,0.00000,0.021128,0.032421,0.002102,0.072335,0.040995,0.022480,0.043143,0.006607,0.001877
mesenchymal_stem_cell_of_adipose_tissue,0.093875,0.000000,0.053223,0.008558,0.029152,0.174913,0.384595,0.000267,0.165017,0.000267,...,0.00000,0.006954,0.029687,0.000267,0.027280,0.244183,0.022733,0.043060,0.011233,0.013907
mast_cell,0.079914,0.000000,0.000535,0.002319,0.001962,0.066536,0.244381,0.000178,0.056547,0.000000,...,0.00000,0.005173,0.012665,0.001427,0.256511,0.004816,0.016054,0.013022,0.002497,0.001605
neutrophil,0.210535,0.000015,0.000059,0.000665,0.000030,0.009613,0.265232,0.000030,0.168316,0.000015,...,0.00000,0.000797,0.002141,0.000428,0.489471,0.000532,0.000591,0.005685,0.000089,0.000281
fibroblast,0.119816,0.000136,0.070691,0.008939,0.017010,0.155285,0.404709,0.000210,0.143152,0.003986,...,0.00005,0.004321,0.035680,0.002167,0.096751,0.216245,0.034528,0.041560,0.004593,0.009545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kidney_epithelial_cell,0.052159,0.002192,0.000877,0.001972,0.000000,0.232851,0.277449,0.006794,0.076594,0.000329,...,0.00011,0.057090,0.023559,0.455950,0.106838,0.154175,0.023449,0.049967,0.018957,0.002849
retinal_pigment_epithelial_cell,0.031250,0.000000,0.000000,0.000000,0.000000,0.078125,0.203125,0.015625,0.062500,0.000000,...,0.00000,0.062500,0.062500,0.000000,0.015625,0.218750,0.000000,0.000000,0.015625,0.031250
keratinocyte,0.071429,0.000000,0.000000,0.000000,0.000000,0.214286,0.214286,0.000000,0.071429,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000
enterocyte_of_epithelium_proper_of_jejunum,0.182716,0.000000,0.000000,0.000000,0.000000,0.219753,0.459259,0.118519,0.148148,0.000000,...,0.00000,0.004938,0.027160,0.227160,0.600000,0.034568,0.017284,0.019753,0.000000,0.012346


In [96]:
# Compute number of cell types each TF shows up in
TF_frac_binarized = (TF_frac > 0.10) * 1
TF_frac_binarized.sum().sort_values(ascending = False)

index
YBX1       175
JUND       175
BPTF       174
NFE2L2     174
KCMF1      174
          ... 
BHLHA9       0
OLIG2        0
POU5F1B      0
LHX5         0
NEUROG2      0
Length: 1637, dtype: int64

## Make TF x Broad Cell Type dataframe of mean expression

In [97]:
if ALL_GENES:
    DATA_CT_TABLE = 'AllGenes/TSP1_30_by_BroadCellClass_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_AllGenes.csv'
else:
    DATA_CT_TABLE = 'TFonly/TSP1_30_by_BroadCellClass_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_TFonly.csv'

In [98]:
CellTypesList = list(adata.obs.broad_cell_class.unique())
len(CellTypesList)

38

In [99]:
TF_df = DataFrame()

for CT in CellTypesList:
    CellsOfThisType = list(adata.obs[adata.obs.broad_cell_class == CT].index)

    CT_adata = adata[CellsOfThisType,TF_list]
    if Raw:
        TF_means = CT_adata.layers['raw_counts'].mean(0)
    else:
        TF_means = CT_adata.layers['log_normalized'].mean(0)

        
    df = pd.DataFrame(TF_means, columns = TF_list, index = [CT])

    TF_df = pd.concat([TF_df,df])



In [100]:
#convert cell type names to indexes without ' ' and '-'
TF_rownames = list(TF_df.index)
TF_rownames = [sub.replace(' ', '_') for sub in TF_rownames]
TF_rownames = [sub.replace('-', '_') for sub in TF_rownames]
TF_rownames = [sub.replace(',', '_') for sub in TF_rownames]
TF_df.index = TF_rownames
TF_df


index,NFYA,ARX,HOXA11,MEOX1,SOX8,ZNF195,ZFX,YBX2,ZNF263,DLX6,...,SEBOX,ZNF280B,ZNF2,HNF1B,DACH1,PCGF2,ZNF670,ZNF8,ZNF229,NR2E3
myeloid_leukocyte,0.148186,0.000342,0.000300,0.002814,0.000805,0.081832,0.469329,0.000313,0.114493,0.000102,...,0.000008,0.006651,0.014110,0.001083,0.188437,0.014667,0.008620,0.019689,0.001755,0.001021
stem_cell,0.114197,0.000687,0.007981,0.003734,0.010814,0.107590,0.337642,0.006231,0.107826,0.000650,...,0.000000,0.004659,0.017681,0.019967,0.128347,0.104125,0.025689,0.020229,0.004671,0.006692
granulocyte,0.262297,0.000016,0.000088,0.000865,0.000261,0.015318,0.347822,0.000042,0.195414,0.000016,...,0.000000,0.001356,0.003400,0.000600,0.996537,0.001074,0.002048,0.006777,0.000280,0.000421
fibroblast,0.081279,0.000095,0.052134,0.006792,0.011205,0.096895,0.321847,0.000147,0.092745,0.001897,...,0.000044,0.002379,0.021265,0.000901,0.111035,0.127676,0.020415,0.022564,0.002692,0.005632
t_cell,0.118937,0.000185,0.000064,0.007764,0.006031,0.123955,0.366441,0.000394,0.103742,0.000070,...,0.000063,0.016754,0.026305,0.000585,0.020851,0.002443,0.006446,0.019346,0.000957,0.001881
contractile_cell,0.093547,0.004619,0.023017,0.007933,0.015198,0.087131,0.319206,0.011565,0.098040,0.004162,...,0.000028,0.018839,0.024757,0.001360,0.217789,0.122939,0.015738,0.028367,0.010766,0.004883
endothelial_cell,0.143818,0.000624,0.000668,0.226397,0.008944,0.122681,0.321601,0.000881,0.126799,0.000244,...,0.000038,0.002676,0.019665,0.000997,0.724881,0.277683,0.026912,0.033539,0.004611,0.005617
erythroid_lineage_cell,0.010743,0.000000,0.000000,0.000250,0.000078,0.004404,0.027605,0.000393,0.008871,0.000000,...,0.000000,0.000472,0.000504,0.000407,0.028678,0.000468,0.001289,0.002883,0.000311,0.000099
innate_lymphoid_cell,0.142996,0.000304,0.000055,0.001912,0.000864,0.156319,0.506997,0.000166,0.127897,0.000078,...,0.000000,0.012236,0.026694,0.001042,0.094030,0.003861,0.006148,0.024552,0.000969,0.001921
lymphocyte_of_b_lineage,0.119203,0.001267,0.000029,0.002020,0.000558,0.122564,0.414547,0.000125,0.097654,0.000023,...,0.000034,0.025396,0.018135,0.000903,0.005138,0.001548,0.016612,0.015815,0.003297,0.001672


In [101]:
# check they are the same 
metaset = set(metadata.broad_cell_class.unique())
TFdfset = set(TF_df.index)
print(TFdfset.difference(metaset))
print(metaset.difference(TFdfset))

set()
set()


In [102]:
print('TFxCT saved as ', DATA_OUT_DIR+DATA_CT_TABLE )
TF_df.to_csv(DATA_OUT_DIR+DATA_CT_TABLE)

TFxCT saved as  /hpc/mydata/robert.jones/TFManuscript2GITHubPush/Results/TFonly/TSP1_30_by_BroadCellClass_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_TFonly.csv


## Make TF x Tissue dataframe of mean expression

In [103]:
if ALL_GENES:
    DATA_CT_TABLE = 'AllGenes/TSP1_30_by_Tissue_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_AllGenes.csv'
else:
    DATA_CT_TABLE = 'TFonly/TSP1_30_by_Tissue_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_TFonly.csv'

In [104]:
CellTypesList = list(adata.obs.tissue.unique())
len(CellTypesList)

28

In [105]:
TF_df = DataFrame()

for CT in CellTypesList:
    CellsOfThisType = list(adata.obs[adata.obs.tissue == CT].index)

    CT_adata = adata[CellsOfThisType,TF_list]
    if Raw:
        TF_means = CT_adata.layers['raw_counts'].mean(0)
    else:
        TF_means = CT_adata.layers['log_normalized'].mean(0)
        
    df = pd.DataFrame(TF_means, columns = TF_list, index = [CT])

    TF_df = pd.concat([TF_df,df])


In [106]:
print('TFxCT saved as ', DATA_OUT_DIR+DATA_CT_TABLE )
TF_df.to_csv(DATA_OUT_DIR+DATA_CT_TABLE)

TFxCT saved as  /hpc/mydata/robert.jones/TFManuscript2GITHubPush/Results/TFonly/TSP1_30_by_Tissue_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_TFonly.csv


In [107]:
# (tissue names have no spaces so they are fine)

### To answer question of why SHOX AND ZBED1 show not mean expression.

In [108]:
adata.var.loc['SHOX', :]

ensembl_id               ENSG00000185960.15
gene_symbol                            SHOX
genome                          Gencode_v41
mt                                    False
ercc                                  False
n_cells_by_counts                         0
mean_counts                             0.0
pct_dropout_by_counts                 100.0
total_counts                            0.0
mean                                    0.0
std                                     1.0
Name: SHOX, dtype: object

In [109]:
adata.var.loc['ZBED1', :]  # this was computed on all assay types,  only ss2 has counts. 

ensembl_id               ENSG00000214717.12
gene_symbol                           ZBED1
genome                          Gencode_v41
mt                                    False
ercc                                  False
n_cells_by_counts                       168
mean_counts                        0.000149
pct_dropout_by_counts              99.99149
total_counts                          295.0
mean                               0.000003
std                                0.000389
Name: ZBED1, dtype: object

In [110]:
adata

View of AnnData object with n_obs × n_vars = 1093048 × 61806
    obs: 'donor', 'tissue', 'anatomical_position', 'method', 'cdna_plate', 'library_plate', 'notes', 'cdna_well', 'old_index', 'assay', 'sample_id', 'replicate', '10X_run', '10X_barcode', 'ambient_removal', 'donor_method', 'donor_assay', 'donor_tissue', 'donor_tissue_assay', 'cell_ontology_class', 'cell_ontology_id', 'compartment', 'broad_cell_class', 'free_annotation', 'manually_annotated', 'published_2022', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ercc', 'pct_counts_ercc', '_scvi_batch', '_scvi_labels', 'scvi_leiden_donorassay_full', 'age', 'sex', 'ethnicity', 'sample_number'
    var: 'ensembl_id', 'gene_symbol', 'genome', 'mt', 'ercc', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: '_scvi_manager_uuid', '_scvi_uuid', '_training_mode', 'age_colors', 'assay_colors', 'compartment_colors', 'donor_colors', 'leiden', 'log1p', 'meth

In [111]:
gene_name = 'ZBED1'

# Get the index of the gene
gene_idx = np.where(adata.var_names == gene_name)[0][0]

# Access the raw_counts layer and sum counts for the gene across all cells
total_counts_zbed1 = adata.X[:, gene_idx].sum()

# Convert to scalar if it’s a numpy matrix or sparse object
if hasattr(total_counts_zbed1, 'A1'):
    total_counts_zbed1 = total_counts_zbed1.A1[0]
elif hasattr(total_counts_zbed1, 'item'):
    total_counts_zbed1 = total_counts_zbed1.item()

print(total_counts_zbed1)

0.0


In [112]:
gene_name = 'SHOX'

# Get the index of the gene
gene_idx = np.where(adata.var_names == gene_name)[0][0]

# Access the raw_counts layer and sum counts for the gene across all cells
total_counts_zbed1 = adata.layers['raw_counts'][:, gene_idx].sum()

# Convert to scalar if it’s a numpy matrix or sparse object
if hasattr(total_counts_zbed1, 'A1'):
    total_counts_zbed1 = total_counts_zbed1.A1[0]
elif hasattr(total_counts_zbed1, 'item'):
    total_counts_zbed1 = total_counts_zbed1.item()

print(total_counts_zbed1)

0


In [113]:
metadata.donor.unique()

array(['TSP27', 'TSP25', 'TSP6', 'TSP14', 'TSP2', 'TSP21', 'TSP12',
       'TSP28', 'TSP1', 'TSP8', 'TSP4', 'TSP15', 'TSP7', 'TSP3', 'TSP26',
       'TSP17', 'TSP30', 'TSP10', 'TSP19', 'TSP20', 'TSP9', 'TSP5',
       'TSP11'], dtype=object)